# Q&A Based Neo4j Database ChatBot using Google'PaLM 2 Model by Bhavesh
[**Link to my YouTube Channel**](https://www.youtube.com/BhaveshBhatt8791?sub_confirmation=1)

# Installs

In [ ]:
!pip install google-generativeai
!pip install -q neo4j-driver
!pip install -q gradio

# Imports

In [ ]:
import google.generativeai as palm
import base64
import json
import gradio as gr
from neo4j import GraphDatabase
import re

# API Key

In [ ]:
palm.configure(api_key = "Enter Your API Key")

# Helper Functions

In [ ]:
def get_answer(input):

    defaults = {
  'model': 'models/text-bison-001',
  'temperature': 0.7,
  'candidate_count': 1,
  'top_k': 40,
  'top_p': 0.95,
  'max_output_tokens': 1024,
  'stop_sequences': [],
  'safety_settings': [{"category":"HARM_CATEGORY_DEROGATORY","threshold":1},{"category":"HARM_CATEGORY_TOXICITY","threshold":1},{"category":"HARM_CATEGORY_VIOLENCE","threshold":2},{"category":"HARM_CATEGORY_SEXUAL","threshold":2},{"category":"HARM_CATEGORY_MEDICAL","threshold":2},{"category":"HARM_CATEGORY_DANGEROUS","threshold":2}],
    }

    prompt = f"""You are an expert in converting English questions to Neo4j Cypher Graph code! The Graph has following Node Labels - Movie, Person and the Movie Node label has the following properties released, tagline, title and the Graph has the following Relationship types ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE!

    All relationships ACTED_IN, DIRECTED, FOLLOWS, PRODUCED, REVIEWED, WROTE start from Person to Movie and not the other way around.

    For example,
    Example 1 - List down 5 movies that released after the year 2000, the Cypher command will be something like this
    ``` MATCH (m:Movie)
    WHERE m.released > 2000
    RETURN m LIMIT 5
    ```

    Example 2 - Get all the people who acted in a movie that was released after 2010.
    ```
    MATCH (p:Person)-[r:ACTED_IN]-(m:Movie)
    WHERE m.released > 2010
    RETURN p,r,m
    ```

    Example 3 - Name the Director of the movie The Matrix Reloaded?
    ```
    MATCH (m:Movie)<-[:DIRECTED]-(p:Person)
    WHERE m.title = 'Apollo 13'
    RETURN p.name
    ```

    Dont include ``` and \n in the output

    {input}"""
    response = palm.generate_text(**defaults, prompt=prompt)
    return response.result

In [ ]:
def extract_query_and_return_key(input_query_result):
    slash_n_pattern = r'[ \n]+'
    ret_pattern = r'RETURN\s+(.*)'
    replacement = ' '

    cleaned_query = re.sub(slash_n_pattern, replacement, input_query_result)
    if cleaned_query:
        match = re.search(ret_pattern, cleaned_query)
        if match:
            extracted_string = match.group(1)
        else:
            extracted_string = ""
    return cleaned_query, extracted_string

In [ ]:
def format_names_with_ampersand(names):
    if len(names) == 0:
        return ""
    elif len(names) == 1:
        return names[0]
    else:
        formatted_names = ", ".join(names[:-1]) + " & " + names[-1]
        return formatted_names

In [ ]:
def run_cypher_on_neo4j(inp_query, inp_key):
    out_list = []
    with driver.session() as session:
        result = session.run(inp_query)
        for record in result:
            out_list.append(record[inp_key])
    driver.close()
    if len(out_list) > 1:
        return format_names_with_ampersand(out_list)
    else:
        return out_list[0]

In [ ]:
def generate_and_exec_cypher(input_query):
    gen_query, gen_key = extract_query_and_return_key(get_answer(input_query))
    return run_cypher_on_neo4j(gen_query, gen_key)

In [ ]:
def chatbot(input, history=[]):
    output = str(generate_and_exec_cypher(input))
    history.append((input, output))
    return history, history

# Initialize Driver

In [ ]:
driver = GraphDatabase.driver("Bolt URL",
                              auth=("Username",
                                    "Password"))

# Launch Gradio Interface

In [ ]:
gr.Interface(fn = chatbot,
             inputs = ["text",'state'],
             outputs = ["chatbot",'state']).launch(debug = True)